In [1]:
import pandas as pd
import numpy as np

import seaborn as sns

from src.tablebuilder import TableBuilder

In [2]:
%load_ext autoreload
%autoreload 2

There are 21,974 rows in the raw dataset, but this in itself is not very useful information - we really want to know how many individual neonates are in the data (some appear on multiple rows, corresponding to, for example, multiple blood culture tests). In theory we should be able to use the unique identifier column (`Uid`) to obtain this information:

In [3]:
data_filepath = './data/sepsis_updated_data_Feb21-Sep24_v2.csv'
raw_df = pd.read_csv(data_filepath)
print('n rows:')
print(len(raw_df))
print('n Uids:')
print(len(raw_df['Uid'].unique()))

n rows:
21974
n Uids:
18839


However, due to technical problems the `Uid` identifier does not appear to be unique, as evidenced by some `Uid`s referring to babies of different genders and admission dates:

In [4]:
(raw_df.groupby('Uid')['Gender'].unique().apply(len) > 1).sum()

220

In [5]:
(raw_df.groupby('Uid')['Datetimeadmission'].unique().apply(len) > 1).sum()

391

Instead we build a separate `id` column, a concatenation of `Uid` and `Datetimeadmission`, which we believe to be remove the problem of duplicated `Uid`s and gives us a total of 19,233 unique cases in the dataset:

In [6]:
raw_df['Datetimeadmission'] = pd.to_datetime(raw_df['Datetimeadmission'])
raw_df['id'] = raw_df['Uid'].str.cat(raw_df['Datetimeadmission'].dt.strftime('%Y-%m-%dT%H:%M:%S'), sep='_')
print('n unique ids:')
print(len(raw_df['id'].unique()))

n unique ids:
19233


In [7]:
len(raw_df.loc[pd.isna(raw_df['Datetimeadmission']), 'id'].unique())

0

Check start and end dates in the dataset:

In [8]:
print('First admission:')
print(raw_df['Datetimeadmission'].min())
print('Last admission:')
print(raw_df['Datetimeadmission'].max())

First admission:
2021-02-02 12:10:00
Last admission:
2024-09-30 19:14:00


Find median birthweight:

In [9]:
raw_df[['id', 'Birthweight']].drop_duplicates()['Birthweight'].str.replace(',', '').astype(float).median()

2700.0

For case fatality rate, we assume that all those who died had the date and time of their death recorded in Neotree:

In [10]:
n_died = len(raw_df.loc[~pd.isna(raw_df['Datetimedeath']), 'id'].unique())
print('n died:', n_died)
n_total = len(raw_df['id'].unique())
print('case fatality rate:', round(n_died / n_total * 1000))

n died: 3097
case fatality rate: 161


How many had blood tests taken?:

In [11]:
raw_df['Neolab_finalbcresult'].value_counts(dropna=False)

Neolab_finalbcresult
NaN                                                       15948
Neg                                                        2653
NegP                                                       1007
Contaminant                                                 977
Pos                                                         529
Negative (FINAL)                                            253
PosP                                                        233
Negative (PRELIMINARY)                                      207
Positive                                                    101
Positive (Preliminary awaiting identification and AST)       46
Possible Contaminant                                         13
Rej                                                           7
Name: count, dtype: int64

In [12]:
print('n cases with test taken:', len(raw_df.loc[~pd.isna(raw_df['Neolab_finalbcresult']), 'id'].unique()))
rejected_values = ['Contaminant', 'Rej', 'Possible Contaminant']
rejected_ids = raw_df.loc[raw_df['Neolab_finalbcresult'].isin(rejected_values), 'id'].unique()
non_rejected_values = [
    'Neg', 'NegP', 'Pos', 'PosP', 'Negative (FINAL)', 'Negative (PRELIMINARY)', 'Positive', 
    'Positive (Preliminary awaiting identification and AST)'
]
non_rejected_ids = raw_df.loc[raw_df['Neolab_finalbcresult'].isin(non_rejected_values), 'id'].unique()
print('n cases with at least one valid test result:', len(non_rejected_ids))
cases_with_no_non_rejected_tests = np.setdiff1d(rejected_ids, non_rejected_ids)
print('n cases with no valid test results:', len(cases_with_no_non_rejected_tests))

n cases with test taken: 4252
n cases with at least one valid test result: 3645
n cases with no valid test results: 607


In [13]:
3645 + 607

4252

In [14]:
raw_df.loc[raw_df['Neolab_finalbcresult'].isin(non_rejected_values), 'Neolab_finalbcresult'].unique()

array(['Pos', 'PosP', 'Neg', 'NegP', 'Positive', 'Negative (PRELIMINARY)',
       'Negative (FINAL)',
       'Positive (Preliminary awaiting identification and AST)'],
      dtype=object)

In [15]:
raw_df.loc[~raw_df['Neolab_finalbcresult'].isin(non_rejected_values), 'Neolab_finalbcresult'].unique()

array([nan, 'Contaminant', 'Possible Contaminant', 'Rej'], dtype=object)

In [16]:
len(raw_df.loc[pd.isna(raw_df['Age']), 'id'].unique())

387

In [17]:
raw_df['Age'] = raw_df['Age'].str.replace(',','').astype(float)

In [18]:
(raw_df['Age'] < 0).sum()

0

In [19]:
(raw_df['Age'] > 72).sum()

556

In [20]:
len(raw_df.loc[raw_df['Age'] > 72, 'id'].unique())

501

In [21]:
501 + 387

888

In [22]:
18345 + 888

19233

Load `datamanager` class to start preparing the raw data for analysis, first looking at the breakdown of the blood test results after removing the rejected rows:

In [23]:
from src.datamanager import DataManager

In [121]:
columns_of_interest = ['Apgar1', 'Apgar5', 'Age', 'Gender',
       'Satsair', 'Typebirth', 'Romlength', 
       'Gestation', 'Birthweight', 'Temperature', 'Skin',
       'Dangersigns', 'Signsrd', 'Wob', 'Activity', 'Umbilicus', 'Colour',
       'Rr', 'Vomiting', 'Abdomen', 'Fontanelle', 'Hr']
data_manager = DataManager(data_filepath)

/hpchome/ccaeelo@IDHS.UCL.AC.UK/projects/neotree-report/src/datamanager.py:94: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df.assign(Neolab_datebct    = pd.to_datetime(df['Neolab_datebct'],    utc=True),
/hpchome/ccaeelo@IDHS.UCL.AC.UK/projects/neotree-report/src/datamanager.py:97: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  Datetimedeath     = pd.to_datetime(df['Datetimedeath'],     utc=True))


18345
18345


How many cases are included in the analysis?

In [122]:
n_included = len(data_manager.df['id'].unique())
print(n_included)

18345


In [123]:
data_manager.df[
    ['id', 'bc_positive_or_diagnosis_or_cause_of_death']
].drop_duplicates()['bc_positive_or_diagnosis_or_cause_of_death'].value_counts(dropna=False)

bc_positive_or_diagnosis_or_cause_of_death
False    17428
True       917
Name: count, dtype: int64

In [124]:
917 + 17428

18345

In [125]:
n_with_diagnosis_recorded = len(data_manager.df.loc[~pd.isna(data_manager.df['Diagdis1']), 'id'].unique())
n_with_eons_diagnosis = len(data_manager.df.loc[data_manager.df['eons_diagnosis'], 'id'].unique())
print('n_with_diagnosis_recorded:', n_with_diagnosis_recorded)
print('n_with_eons_diagnosis:', n_with_eons_diagnosis)
print(f'pct of diagnoses that were EONS: {n_with_eons_diagnosis / n_with_diagnosis_recorded * 100:.3f}')

n_with_diagnosis_recorded: 13753
n_with_eons_diagnosis: 489
pct of diagnoses that were EONS: 3.556


In [126]:
n_with_death_recorded = len(data_manager.df.loc[~pd.isna(data_manager.df['Causedeath']), 'id'].unique())
n_with_death_recorded_as_eons = len(data_manager.df.loc[data_manager.df['eons_cause_of_death'], 'id'].unique())
print('n_with_death_recorded:', n_with_death_recorded)
print('n_with_death_recorded_as_eons:', n_with_death_recorded_as_eons)
print(f'pct of causes of death that were recorded as EONS: {n_with_death_recorded_as_eons / n_with_death_recorded * 100:.3f}')

n_with_death_recorded: 2926
n_with_death_recorded_as_eons: 175
pct of causes of death that were recorded as EONS: 5.981


In some cases, the results of the blood test are not material to our modelling strategy, as a neonate testing negative with a positive diagnosis would still be counted as positive, because of the diagosis. The same is true for cause of death:

In [127]:
len(data_manager.df.loc[(~pd.isna(data_manager.df['Neolab_finalbcresult'])) &
                        (data_manager.df['eons_diagnosis'] | data_manager.df['eons_cause_of_death']), 'id'].unique())

334

In [128]:
len(data_manager.df.loc[(data_manager.df['eons_diagnosis'] | data_manager.df['eons_cause_of_death']), 'id'].unique())

664

In [129]:
489 + 175

664

In [130]:
len(data_manager.df.loc[(data_manager.df['eons_diagnosis'] & data_manager.df['eons_cause_of_death']), 'id'].unique())

0

If we remove duplicate rows in the data, we can see how an explanation for the construction of the composite outcome variable breaks down:

In [131]:
data_manager.remove_duplicate_predictors(columns_of_interest, 'bc_positive_or_diagnosis_or_cause_of_death')
data_manager.df['description'].value_counts()

description
no_tests_taken        13938
neg_result_found       1605
all_tests_excluded     1548
diagnosis_or_death      664
confusing               337
pos_result_found        253
Name: count, dtype: int64

In [132]:
13938 + 1548 + 1605 + 337

17428

In [133]:
253 + 664

917

In [134]:
data_manager.df['bc_positive_or_diagnosis_or_cause_of_death'].value_counts()

bc_positive_or_diagnosis_or_cause_of_death
False    17428
True       917
Name: count, dtype: int64

For the analysis of missing values:

In [135]:
all_features = ['Apgar1', 'Apgar5', 'Apgar10', 'Age', 'Gender',
       'Bsmmol', 'Satsair', 'Typebirth', 'Romlength',
       'Gestation', 'Birthweight', 'Temperature', 'Skin',
       'Dangersigns', 'Signsrd', 'Wob', 'Activity', 'Umbilicus', 'Colour',
       'Rr', 'Vomiting', 'Abdomen', 'Fontanelle', 'Hr']
is_float = data_manager.df[all_features].dtypes == 'float64'

for index, value in data_manager.df[np.array(all_features)[is_float]].isna().sum().sort_values(ascending=False).items(): 
       print(f'{index:<12} | {value:>5} | {value / len(data_manager.df):.3%}')

Bsmmol       | 18098 | 98.654%
Apgar10      | 16528 | 90.095%
Apgar5       |  1227 | 6.688%
Apgar1       |  1183 | 6.449%
Satsair      |   341 | 1.859%
Rr           |   140 | 0.763%
Hr           |    93 | 0.507%
Birthweight  |    54 | 0.294%
Age          |     0 | 0.000%
Gestation    |     0 | 0.000%
Temperature  |     0 | 0.000%


In [136]:
for index, value in (data_manager.df[np.array(all_features)[-is_float]] == 'missing').sum().sort_values(ascending=False).items(): 
       print(f'{index:<12} | {value:>5} | {value / len(data_manager.df):.3%}')

Skin         | 18190 | 99.155%
Dangersigns  | 13093 | 71.371%
Wob          |  9860 | 53.748%
Signsrd      |  9303 | 50.711%
Romlength    |  1338 | 7.294%
Abdomen      |    49 | 0.267%
Vomiting     |    27 | 0.147%
Colour       |    24 | 0.131%
Umbilicus    |    22 | 0.120%
Typebirth    |     0 | 0.000%
Gender       |     0 | 0.000%
Activity     |     0 | 0.000%
Fontanelle   |     0 | 0.000%


To count values in the composite outcome variable with missing continuous values removed:

In [139]:
X_train, X_test, y_train, y_test = data_manager.get_X_y(columns_of_interest, seed=2024, y_label='bc_positive_or_diagnosis_or_cause_of_death')
y = pd.concat([y_train, y_test])
print('Total number of rows:', len(y))
pd.Series(y).value_counts()

Total number of rows: 16595


bc_positive_or_diagnosis_or_cause_of_death
False    15790
True       805
Name: count, dtype: int64

Finally, we generate the values for the table summarising the input data, using some fairly complex logic hidden in `./src/tablebuilder.py`:

In [148]:
table_builder = TableBuilder()
table_df = table_builder.run(data_manager)
table_df.to_csv('./output/table_df_20250207_v2.csv', index=False, header=[
    'Predictor', 'Levels', 'All cases (n=18,345)', 'Composite outcome positive (n=917)', 'Composite outcome negative (n=17,428)'
])
table_df

male
female
not_sure
twin
single
triplet
missing
noprom
prom
missing
condition_present
missing
mild
moderate
severe
missing
signs_present
missing
signs_present
alert
lethargic
coma
irritable
convulsions
Pink
Blue
White
Yell
missing
{Pink,White}
{Yell,White}
Norm
Abnormal
missing
No
Yes
Poss
missing
Value(data_name='*', display_name='Yes (all feeds/ blood/ green)')
Norm
Abnormal
missing
flat
bulging
sunken


,0,1,2,3,4
0,Age (hour); median [Q1-Q3],-,2.0 [1.0-7.0],3.0 [1.0-18.0],2.0 [1.0-6.0]
1,Gender; n (%),Female,8222 (44.82),399 (43.51),7823 (44.89)
2,,Male,10089 (55.00),516 (56.27),9573 (54.93)
3,,Ambiguous,34 (0.19),2 (0.22),32 (0.18)
4,Gestational age; median [Q1-Q3],-,38 [35-40],38 [34-39],38 [35-40]
5,Birth weight; median [Q1-Q3],-,2710 [2000-3200],2700 [1850-3200],2725 [2000-3200]
6,,Missing,54 (0.29),2 (3.70),52 (96.30)
7,Temperature at admission; median [Q1-Q3],-,36.8 [36.6-37.0],36.8 [36.6-37.0],36.8 [36.6-37.0]
8,Heart rate; median [Q1-Q3],-,138.0 [129.0-147.0],140.0 [129.0-150.0],138.0 [129.0-146.0]
9,,Missing,93 (0.51),3 (3.23),90 (96.77)
